# Convlstm D + Plot loss

In [1]:
from __future__ import print_function, division


# from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D, ConvLSTM2D, Conv3D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import tensorflow as tf

import matplotlib.pyplot as plt

import sys

import numpy as np

import os


In [3]:
class MIGAN():
    def __init__(self):
        self.frames = 3
        self.img_rows = 64 #28
        self.img_cols = 64 #28
        self.channels = 1 #1
        self.version = "2.4"
        self.train_version = "a"
        
        # train as sequence with 3 frames
        self.seq_shape = (self.frames, self.img_rows, self.img_cols, self.channels)


        # Cut and load the dataset to shape (90000,3,64,64,1)
        data = np.load("mnist_test_seq.npy")
        train_set = np.concatenate((data[0:6],data[6:12],data[12:18],data[1:7],data[7:13],data[13:19],data[8:14],data[14:20]),axis=1)
        test_set = data[2:8]
        
        self.Y_train = np.expand_dims(train_set[3:6].transpose(1,0,2,3), axis=4)
        self.X_train = np.expand_dims(train_set[0:3].transpose(1,0,2,3), axis=4)
        
        self.Y_test = np.expand_dims(test_set[3:6].transpose(1,0,2,3), axis=4)
        self.X_test = np.expand_dims(test_set[0:3].transpose(1,0,2,3), axis=4)
        

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes ipt_imgs as input and generates gen_imgs
        ipt_imgs = Input(shape=(self.seq_shape))
        gen_imgs = self.generator(ipt_imgs)
        
        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(gen_imgs)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(ipt_imgs, validity)
        self.combined.compile(loss=['mean_squared_error','binary_crossentropy'],loss_weights=[0.999,0.001],optimizer=optimizer)
#         self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)
        
        # Set checkpoints and save trained models
        self.checkpoint_dir = 'training_checkpoints' + self.version
        self.checkpoint_prefix = os.path.join(self.checkpoint_dir, "ckpt")
        self.checkpoint = tf.train.Checkpoint(generator_optimizer=optimizer,
                                         discriminator_optimizer=optimizer,
                                         generator=self.generator,
                                         discriminator=self.discriminator)

        
    def build_generator(self):


        model_convlstm = Sequential(
            [
                ConvLSTM2D(filters=40, kernel_size=(3, 3), padding="same", return_sequences=True, input_shape=self.seq_shape),
                BatchNormalization(),
                ConvLSTM2D(filters=40, kernel_size=(3, 3), padding="same", return_sequences=True),
                BatchNormalization(),
                ConvLSTM2D(filters=40, kernel_size=(3, 3), padding="same", return_sequences=True),
                BatchNormalization(),
                ConvLSTM2D(filters=40, kernel_size=(3, 3), padding="same", return_sequences=True),
                BatchNormalization(),
                Conv3D(filters=1, kernel_size=(3, 3, 3), activation="sigmoid", padding="same"),
            ])

        model_convlstm.summary()
        


        ipt_imgs = Input(shape=(self.seq_shape))
        gen_imgs = model_convlstm(ipt_imgs)

        return Model(ipt_imgs, gen_imgs)

    def build_discriminator(self):

        model = Sequential(
            [
                ConvLSTM2D(filters=40, kernel_size=(3, 3), padding="same", return_sequences=True, input_shape=self.seq_shape),
                BatchNormalization(),
                ConvLSTM2D(filters=40, kernel_size=(3, 3), padding="same", return_sequences=True),
                BatchNormalization(),
                ConvLSTM2D(filters=40, kernel_size=(3, 3), padding="same", return_sequences=True),
                BatchNormalization(),
                Conv3D(filters=1, kernel_size=(3, 3, 3), activation="sigmoid", padding="same"),       
            ])
        model.add(Flatten())
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        
        model.summary()
        

        sam_imgs = Input(shape=self.seq_shape)
        validity = model(sam_imgs)

        return Model(sam_imgs, validity)

    def train(self, epochs, batch_size , sample_interval):
        # prepare lists for storing stats each iteration
        d1_hist, d2_hist, g_hist, mse_hist= list(), list(), list(), list()

        # Rescale -1 to 1
        Y_train = self.Y_train / 127.5 - 1.0
        X_train = self.X_train / 127.5 - 1.0
        
        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        
        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, Y_train.shape[0], batch_size) 
            sam_imgs = Y_train[idx] #For Y_train
            ipt_imgs = X_train[idx]
            
            # Generate a batch of new images            
            gen_imgs = self.generator.predict(ipt_imgs)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(sam_imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            MSE_loss = self.evaluate()

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (to have the discriminator label samples as valid)
            g_loss = self.combined.train_on_batch(ipt_imgs, valid)

            # Plot the progress every 20 epochs
            if (epoch + 1) % 10 == 0:
                print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f] [MSE loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss, MSE_loss))
            
            # Save the models every 60 epochs
            if (epoch + 1) % 60 == 0:
                self.checkpoint.save(file_prefix = self.checkpoint_prefix)
                
            
            # If at save interval => save generated image samples
            if epoch % 20 == 0:
                self.save_images(epoch)
                
            # Record history
            d1_hist.append(d_loss[0])
            d2_hist.append(d_loss[1])
            g_hist.append(g_loss)
            mse_hist.append(MSE_loss)
            
            if epoch % 100 == 0:
                self.plot_history(d1_hist, d2_hist, g_hist, mse_hist, epoch)

                

    def save_images(self, epoch):
        saveimg_dir = "generated_images"+self.version +"/%s.png"
        save_name = self.train_version + str(epoch)
        
        # Select Y_train and X_train
        Y_train = self.Y_train / 127.5 - 1.0
        X_train = self.X_train / 127.5 - 1.0
        
        # Select a clip for ploting
        idx = np.random.randint(0, Y_train.shape[0], 32)
        ipt_imgs = X_train[idx][0].squeeze()
        gen_imgs = self.generator.predict(X_train[idx])[0].squeeze()
        sam_imgs = Y_train[idx][0].squeeze()
        
        # Plot images
        fig = plt.figure()
        row1 = plt.subplot(3,3,1)
        plt.imshow(sam_imgs[0,:,:], cmap='gray')
        row1.title.set_text("Target sequence")
        plt.axis("off")
        plt.subplot(3,3,2)
        plt.imshow(sam_imgs[1,:,:], cmap='gray')
        plt.axis("off")
        plt.subplot(3,3,3)
        plt.imshow(sam_imgs[2,:,:], cmap='gray')
        plt.axis("off")

        row2 = plt.subplot(3,3,4)
        plt.imshow(ipt_imgs[0,:,:], cmap='gray')
        row2.title.set_text("Input sequence")
        plt.axis("off")
        plt.subplot(3,3,5)
        plt.imshow(ipt_imgs[1,:,:], cmap='gray')
        plt.axis("off")
        plt.subplot(3,3,6)
        plt.imshow(ipt_imgs[2,:,:], cmap='gray')
        plt.axis("off")
                
        row2 = plt.subplot(3,3,7)
        plt.imshow(gen_imgs[0,:,:], cmap='gray')
        row2.title.set_text("Generated sequence")
        plt.axis("off")
        plt.subplot(3,3,8)
        plt.imshow(gen_imgs[1,:,:], cmap='gray')
        plt.axis("off")
        plt.subplot(3,3,9)
        plt.imshow(gen_imgs[2,:,:], cmap='gray')
        plt.axis("off")
    
        fig.savefig(saveimg_dir % save_name) #%d
        plt.close()

#         plt.show()

    def plot_history(self, d1_hist, d2_hist, g_hist, mse_hist, epoch):
        saveimg_dir = "generated_images"+self.version +"/%s.png"
        save_name = "loss" + self.train_version + str(epoch+120) 
        
        # plot loss
        plt.plot(d1_hist, label='d-real')
        plt.plot(d2_hist, label='d-fake')
        plt.plot(g_hist, label='gen')
        plt.plot(mse_hist, label='mse')
        plt.legend()
        
        plt.savefig(saveimg_dir % save_name)
        plt.close()

#         plt.show


    def evaluate(self):
        avg_mse = 0
        Y_test = self.Y_test / 127.5 - 1.0
        X_test = self.X_test / 127.5 - 1.0
        
        # Select a clip
        idx = np.random.randint(0, Y_test.shape[0], 32)
        ipt_sequences = X_test[idx].squeeze()
        gen_sequences = self.generator.predict(X_test[idx]).squeeze()
        tgt_sequences = Y_test[idx].squeeze()
        
        avg_mse += tf.reduce_mean(tf.math.squared_difference(tgt_sequences, gen_sequences)).numpy()
#         print(avg_mse)
        return avg_mse
                            
                                    
if __name__ == '__main__':
    gan = MIGAN()
    gan.checkpoint.restore(tf.train.latest_checkpoint(gan.checkpoint_dir))
    gan.train(epochs=10000, batch_size=32, sample_interval=20)
    

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_7 (ConvLSTM2D)  (None, 3, 64, 64, 40)     59200     
_________________________________________________________________
batch_normalization_7 (Batch (None, 3, 64, 64, 40)     160       
_________________________________________________________________
conv_lst_m2d_8 (ConvLSTM2D)  (None, 3, 64, 64, 40)     115360    
_________________________________________________________________
batch_normalization_8 (Batch (None, 3, 64, 64, 40)     160       
_________________________________________________________________
conv_lst_m2d_9 (ConvLSTM2D)  (None, 3, 64, 64, 40)     115360    
_________________________________________________________________
batch_normalization_9 (Batch (None, 3, 64, 64, 40)     160       
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 3, 64, 64, 1)     